In [202]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import backtrader as bt
from datetime import datetime
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint

In [195]:
soxx_data = yf.download("SOXX", start='2021-01-01')
nvda_data = yf.download("NVDA", start='2021-01-01')
tsmc_data = yf.download("TSM", start='2021-01-01')
intc_data = yf.download("INTC", start='2021-01-01')
nxpi_data = yf.download("NXPI", start='2021-01-01')
avgo_data = yf.download("AVGO", start='2021-01-01')
txn_data = yf.download("TXN", start='2021-01-01')
amd_data = yf.download("AMD", start='2021-01-01')
data_list = [soxx_data, nvda_data, tsmc_data, intc_data, nxpi_data, avgo_data, txn_data, amd_data]
data_list_names = ["soxx_data", "nvda_data", "tsmc_data", "intc_data", "nxpi_data", "avgo_data", "txn_data", "amd_data"]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [153]:
soxx_data["Close"].corr(nvda_data["Close"])

0.9488188446068953

In [154]:
soxx_data["Close"].corr(tsmc_data["Close"])

-0.003757109863184324

In [155]:
soxx_data["Close"].corr(intc_data["Close"])

-0.5278427450864056

In [156]:
soxx_data["Close"].corr(nxpi_data["Close"])

0.8022136853111411

In [157]:
soxx_data["Close"].corr(avgo_data["Close"])

0.9453770724621768

In [212]:
pval = 1
single_pval = 1
coint_val = 1
combo = 2
for i1, ticker1 in enumerate(data_list):
    single_stats = adfuller(ticker1["Close"])
    if single_stats[1] < single_pval:
        single_pval = single_stats[1]
        single_min_ticker = data_list_names[i1]
        single_min = single_stats
    for i2, ticker2 in enumerate(data_list[i1+1:]):
        coint_res = coint(ticker1["Close"], ticker2["Close"])
        if coint_res[1] < coint_val:
            coint_val = coint_res[1]
            coint_pair = data_list_names[i1] + " " + data_list_names[i1+i2+1]
        model = sm.OLS(ticker1["Close"], ticker2["Close"]).fit()
        test_res = adfuller(model.resid)
        if test_res[1] < pval:
            pval = test_res[1]
            pair_tickers = data_list_names[i1] + " " + data_list_names[i1+i2+1]
            pair = test_res
        if test_res[1] + coint_res[1] < combo:
            combo = test_res[1] + coint_res[1]
            combo_tickers = data_list_names[i1] + " " + data_list_names[i1+i2+1]
        
print(single_min_ticker)
print(single_min)
print(" ")
print(pair_tickers)
print(pair)
print(" ")
print(coint_pair)
print(coint_val)   
print(" ")
print(combo_tickers)
print(combo)
    

txn_data
(-2.985146098796241, 0.03630504539965115, 1, 229, {'1%': -3.4592326027153493, '5%': -2.8742454699025872, '10%': -2.5735414688888465}, 1090.7400906538635)
 
nxpi_data txn_data
(-2.738571606609665, 0.06759819572370056, 5, 225, {'1%': -3.4597521044060353, '5%': -2.874472927517147, '10%': -2.5736628197530864}, 1069.3197966495354)
 
avgo_data amd_data
0.0011199906533953213
 
nxpi_data txn_data
0.2688394803698743


In [220]:
ratio = txn_data["Close"] / nxpi_data["Close"]
z_score = (ratio - ratio.mean()) / np.std(ratio)
std_up = np.mean(ratio) + np.std(z_score)
std_down = np.mean(ratio) - np.std(z_score)

In [225]:
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind


class MultiDataStrategy(bt.Strategy):
    '''
    This strategy operates on 2 datas. The expectation is that the 2 datas are
    correlated and the 2nd data is used to generate signals on the 1st

      - Buy/Sell Operationss will be executed on the 1st data
      - The signals are generated using a Simple Moving Average on the 2nd data
        when the close price crosses upwwards/downwards

    The strategy is a long-only strategy
    '''
    params = dict(
        period=15,
        stake=10,
        printout=True,
    )

    def log(self, txt, dt=None):
        if self.p.printout:
            dt = dt or self.data.datetime[0]
            dt = bt.num2date(dt)
            print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [bt.Order.Submitted, bt.Order.Accepted]:
            return  # Await further notifications

        if order.status == order.Completed:
            if order.isbuy():
                buytxt = 'BUY COMPLETE, %.2f' % order.executed.price
                self.log(buytxt, order.executed.dt)
            else:
                selltxt = 'SELL COMPLETE, %.2f' % order.executed.price
                self.log(selltxt, order.executed.dt)

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            self.log('%s ,' % order.Status[order.status])
            pass  # Simply log

        # Allow new orders
        self.orderid = None

    def __init__(self):
        # To control operation entries
        self.orderid = None

        # Create SMA on 2nd data
        ratio = self.data0 / self.data1
        mean = bt.ind.SMA(ratio)
        z_score = (ratio - mean) / bit.ind.stddev(ratio)
        # Create a CrossOver Signal from close an moving average
        top = bt.ind.CrossUp(z_score, 1)
        bottom = bt.ind.CrossUp(z_score, -1)

    def next(self):
        if self.orderid:
            return  # if an order is active, no new orders are allowed

        if not self.position:  # not yet in market
            if self.signal > 0.0:  # cross upwards
                self.log('BUY CREATE , %.2f' % self.data1.close[0])
                self.buy(size=self.p.stake)

        else:  # in the market
            if self.signal < 0.0:  # crosss downwards
                self.log('SELL CREATE , %.2f' % self.data1.close[0])
                self.sell(size=self.p.stake)

    def stop(self):
        print('==================================================')
        print('Starting Value - %.2f' % self.broker.startingcash)
        print('Ending   Value - %.2f' % self.broker.getvalue())
        print('==================================================')


def runstrategy():
    args = parse_args()

    # Create a cerebro
    cerebro = bt.Cerebro()

    # Get the dates from the args
    fromdate = datetime.datetime.strptime(args.fromdate, '%Y-%m-%d')
    todate = datetime.datetime.strptime(args.todate, '%Y-%m-%d')

    # Create the 1st data
    data0 = btfeeds.YahooFinanceCSVData(
        dataname=args.data0,
        fromdate=fromdate,
        todate=todate)

    # Add the 1st data to cerebro
    cerebro.adddata(data0)

    # Create the 2nd data
    data1 = btfeeds.YahooFinanceCSVData(
        dataname=args.data1,
        fromdate=fromdate,
        todate=todate)

    # Add the 2nd data to cerebro
    cerebro.adddata(data1)

    # Add the strategy
    cerebro.addstrategy(MultiDataStrategy,
                        period=args.period,
                        stake=args.stake)

    # Add the commission - only stocks like a for each operation
    cerebro.broker.setcash(args.cash)

    # Add the commission - only stocks like a for each operation
    cerebro.broker.setcommission(commission=args.commperc)

    # And run it
    cerebro.run()

    # Plot if requested
    if args.plot:
        cerebro.plot(numfigs=args.numfigs, volume=False, zdown=False)


def parse_args():
    parser = argparse.ArgumentParser(description='MultiData Strategy')

    parser.add_argument('--data0', '-d0',
                        default='../../datas/orcl-1995-2014.txt',
                        help='1st data into the system')

    parser.add_argument('--data1', '-d1',
                        default='../../datas/yhoo-1996-2014.txt',
                        help='2nd data into the system')

    parser.add_argument('--fromdate', '-f',
                        default='2003-01-01',
                        help='Starting date in YYYY-MM-DD format')

    parser.add_argument('--todate', '-t',
                        default='2005-12-31',
                        help='Starting date in YYYY-MM-DD format')

    parser.add_argument('--period', default=15, type=int,
                        help='Period to apply to the Simple Moving Average')

    parser.add_argument('--cash', default=100000, type=int,
                        help='Starting Cash')

    parser.add_argument('--commperc', default=0.005, type=float,
                        help='Percentage commission for operation (0.005 is 0.5%%')

    parser.add_argument('--stake', default=10, type=int,
                        help='Stake to apply in each operation')

    parser.add_argument('--plot', '-p', action='store_true',
                        help='Plot the read data')

    parser.add_argument('--numfigs', '-n', default=1,
                        help='Plot using numfigs figures')

    return parser.parse_args()


if __name__ == '__main__':
    runstrategy()

Date
2021-01-04    1.525516
2021-01-05    1.179297
2021-01-06    0.873504
2021-01-07    0.553941
2021-01-08    0.980543
                ...   
2021-11-24   -1.413029
2021-11-26   -1.172741
2021-11-29   -1.598404
2021-11-30   -1.756701
2021-12-01   -1.788267
Name: Close, Length: 231, dtype: float64